# Lab 1
In this lab, you will learn to make API Requests using Python.  The API in this lab is from the [FourSquare](www.foursquare.com) application. Check out the website and understand some of the features FourSquare has to offer.


## Step 1:  Setting up a Developer Account
Go to http://developer.foursquare.com and click "Create New" to set up a new developer account. Once your account is set up, go ahead and make a new app.  Name the App "MEETLab7" and use http://meet.mit.edu as the website. You can leave all the other fields blank and click "Save Changes" when you are done. The next page will give you a foursquare client id and client secret, go ahead and paste those values here, we will need them to access Foursquare's API. ![Foursqure Developer Account](FOURSQUARE1.PNG)

In [ ]:
foursquare_client_id = "PASTE YOUR CLIENT ID HERE"
foursquare_client_secret = "PASTE YOUR CLIENT SECRET HERE"

## Step 2: Exploring the API Documentation
If you check out https://developer.foursquare.com/docs/ you are taken to the documentation for Foursquare's API.  In this lab, we are going to try to use the [Search](https://developer.foursquare.com/docs/venues/search) feature to see if we can find some nice restaurants to have dinner.  

In the API documentation we can see that in order to search the api there are few things that are required:

* The URL of the API we are going to access in this case it is "https://api.foursquare.com/v2/venues/search"

* A "?" is added next in the URL to indicate that we are passing in parameters, then each parameter is separated by a "&" sign

* We must include our client id and client secret in the request

* A version of the API you will be accessing (for today we will use version "20150603")


* If we are searching around a city we must add a "near" parameter

* If we are searching for a specific type of location (such as a Sushi Bar) we must add a "query" parameter (also note that phrases with spaces are replaced with the "+" symbol)


Putting all of these together we get an API url that looks something like this:

```
https://api.foursquare.com/v2/venues/search?client_id=YOURCLIENTID&client_secret=YOURCLIENTSECRET&v=20130815&near=%San+Francisco&query=Sushi+Bar
```

we can create a URL string in Python that takes in the parameters as python variables like so:

In [ ]:
location = "Dallas"
mealType = "Sushi"
version = "20150603"
url = ('https://api.foursquare.com/v2/venues/search?client_id=%s&client_secret=%s&v=%s&near=%s&query=%s' % (foursquare_client_id, foursquare_client_secret,version,location,mealType))


# Step 3 Sending an API Request in Python
There are many ways to make API requests in Python.  In this lab we will be using the [Requests](docs.python-requests.org/) library in Python
__ you may need to run ```sudo pip install requests``` to use the this library __

In [ ]:
import requests

result = requests.get(url).json()

This line of code will send a GET request to the URL we provided and store the answer in a JSON file.  Let's see what is inside the response below:

In [ ]:
result

Whoa,that was gross! What is all this information?!?! Well, Fourquare returned all of the results it could find inside a JSON string. But how do we just get the first result out of the JSON string?

Well, JSON strings can be treated like dictionaries in python.  What happens when we look at the keys of the result?

In [ ]:
result.keys()

So the result contains "Meta" which contains all the metadata. This is just information about the request we sent.  The "Response" key contains the actual information we requested. so let's see what's inside result['response']

In [ ]:
result['response']

Seeing that this data is contained inside curly braces "{ }" means that we have another response dictionary inside of our original result dictionary.  Let's see what keys it contains

In [ ]:
result['response'].keys()

Venues are the places that our request returned. Let's see what's inside of it.

In [ ]:
result['response']['venues']

Now, we have data contained inside of brackets "[]". So this is a list of data.  This list contains all of the venues that matched our query.  For this lab, we are only interested in finding the first match, so let's grab the 0th index item from the list

In [ ]:
result['response']['venues'][0]

And it seems we have another dictionary inside of this first venue.  So let's check the keys again.

In [ ]:
result['response']['venues'][0].keys()

Ok this looks promising. What happens if I look inside the "name" key?

In [ ]:
result['response']['venues'][0]['name']

Great! Let's store this as a variable

In [ ]:
restaurant_name = result['response']['venues'][0]['name']

Now, let's see what's inside the location key:

In [ ]:
result['response']['venues'][0]['location']

That's a lot of information about the address. The formatted address contains all of the information we would need to find the location, but it is inside of another list.  Maybe we can write a for loop to store it all in one string.

In [ ]:
restaurant_address = ""
for i in restaurant['location']['formattedAddress']:
    restaurant_address += i + " "

Now let's see how our address looks:

In [ ]:
restaurant_address

# Step 4. Putting it all together

Ok cool. Now let's see if we can put it all together in one function. But before that, be sure to modify your location and string inputs to substitute spaces with "+" symbols

In [ ]:
testString = "This is a test string"
testString.replace(" ","+")

In [ ]:
def findARestaurant(mealType, location):
    mealType = mealType.replace(" ", "+")
    location = location.replace(" ", "+")
    version = "20150603"
    url = ('https://api.foursquare.com/v2/venues/search?client_id=%s&client_secret=%s&v=%s&near=%s&query=%s' % (foursquare_client_id, foursquare_client_secret,version,location,mealType))
    result = requests.get(url).json()
    #Make sure the request returned venues
    if len(result['response']['venues']) > 0:
        restaurant_name = result['response']['venues'][0]['name']
        #Also, check that the restaurant has a provided address
        if 'address' in result['response']['venues'][0]['location'].keys():
            restaurant_address = result['response']['venues'][0]['location']['address']
        else: 
            restaurant_address = "No Address Provided"
        print(restaurant_name, restaurant_address)
    else:
        print("No Restaurants Found")
    


In [ ]:
findARestaurant("Sushi", "Tel Aviv")

In [ ]:
findARestaurant("Ribs", "Beit Jala")

In [ ]:
findARestaurant("Falafel", "Jerusalem")

Now try finding restaurants all over the world!

In [ ]:
findARestaurant("Pizza", "Tokyo Japan")
findARestaurant("Tacos", "Jakarta Indonesia")
findARestaurant("Tapas", "Maputo Mozambique")
findARestaurant("Falafel", "Cairo Egypt")
findARestaurant("Spaghetti", "New Delhi India")
findARestaurant("Cappuccino", "Geneva Switzerland")
findARestaurant("Sushi", "Los Angeles California")
findARestaurant("Steak", "Santiago Chile")
findARestaurant("Gyros", "Sydney Australia")

## Step 4: Finding Photos 
Now that you've found the name and address of restarants. Let's explore the [Foursquare Venue Photos API](https://developer.foursquare.com/docs/venues/photos) to see if we can find images for our results. In order to search for photos we must get the unique venue id from our previous request and pass it into the API of our new URL.

In [ ]:
mealType = "Salad"
location = "Istanbul+Turkey"
version = "20150603"
url = ('https://api.foursquare.com/v2/venues/search?client_id=%s&client_secret=%s&v=%s&near=%s&query=%s' % (foursquare_client_id, foursquare_client_secret,version,location,mealType))
result = requests.get(url).json()
restaurant_name = result['response']['venues'][0]['name']
restaurant_id = result['response']['venues'][0]['id']
print(restaurant_name, restaurant_id)

Now that we have the id we can pass it into our new API request to find images:

In [ ]:
url = ('https://api.foursquare.com/v2/venues/%s/photos?client_id=%s&v=20150603&client_secret=%s' % ((restaurant_id,foursquare_client_id,foursquare_client_secret)))
result = requests.get(url).json()
result

I'll save you some time exploring this JSON response and tell you where you can find the first photo:

In [ ]:

firstpic = result['response']['photos']['items'][0]
firstpic

Images on Foursquare's API are broken into prefixes and suffixes.  Between the two we can insert the dimensions we want for our picture. Let's size the picture to 300x300 and display it:

In [ ]:
from IPython.display import Image


prefix = firstpic['prefix']
suffix = firstpic['suffix']
imageURL = prefix + "300x300" + suffix

Image(url=imageURL)

Yum Yum!  Now create a findARestaurants function, that returns a resturant name, address & picture for a restaurant in a given location.  Make sure your function is able to handle:
* The API not finding a restaurant
* The API not finding an address
* The API not finding an image
* The user specifying inputs with punctuation marks or capitalization differences. 

In [ ]:
def findARestaurant(mealType,location):
    pass #<--put your code here and erase 'pass'

In [ ]:
findARestaurant("Pizza", "Tokyo Japan")
findARestaurant("Tacos", "Jakarta Indonesia")
findARestaurant("Tapas", "Maputo Mozambique")
findARestaurant("Falafel", "Cairo Egypt")
findARestaurant("Spaghetti", "New Delhi India")
findARestaurant("Cappuccino", "Geneva Switzerland")
findARestaurant("Sushi", "Los Angeles California")
findARestaurant("Steak", "Santiago Chile")
findARestaurant("Gyros", "Sydney Australia")